In [ ]:
import flet as ft
import requests
import sqlite3
from datetime import datetime

AREA_URL = "http://www.jma.go.jp/bosai/common/const/area.json"
FORECAST_URL = "https://www.jma.go.jp/bosai/forecast/data/forecast/{}.json"
DB_NAME = "weather.db"


# ---------- DB初期化 ----------
def init_db():
    conn = sqlite3.connect(DB_NAME)
    cur = conn.cursor()

    cur.execute("""
        CREATE TABLE IF NOT EXISTS areas (
            area_code TEXT PRIMARY KEY,
            area_name TEXT
        )
    """)

    cur.execute("""
        CREATE TABLE IF NOT EXISTS forecasts (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            area_code TEXT,
            report_date TEXT,
            target_date TEXT,
            weather TEXT,
            FOREIGN KEY(area_code) REFERENCES areas(area_code)
        )
    """)

    conn.commit()
    conn.close()


# ---------- エリア保存 ----------
def save_areas(offices):
    conn = sqlite3.connect(DB_NAME)
    cur = conn.cursor()

    for code, info in offices.items():
        cur.execute(
            "INSERT OR IGNORE INTO areas VALUES (?, ?)",
            (code, info["name"])
        )

    conn.commit()
    conn.close()


# ---------- 天気保存 ----------
def save_forecast(area_code, forecast_json):
    conn = sqlite3.connect(DB_NAME)
    cur = conn.cursor()

    report_date = forecast_json[0]["reportDatetime"]

    for ts in forecast_json[0]["timeSeries"][0]["areas"]:
        for date, weather in zip(
            forecast_json[0]["timeSeries"][0]["timeDefines"],
            ts["weathers"]
        ):
            cur.execute("""
                INSERT INTO forecasts (area_code, report_date, target_date, weather)
                VALUES (?, ?, ?, ?)
            """, (area_code, report_date, date, weather))

    conn.commit()
    conn.close()


# ---------- DBから天気取得 ----------
def load_forecast(area_code):
    conn = sqlite3.connect(DB_NAME)
    cur = conn.cursor()

    cur.execute("""
        SELECT target_date, weather
        FROM forecasts
        WHERE area_code = ?
        ORDER BY target_date
    """, (area_code,))

    rows = cur.fetchall()
    conn.close()
    return rows


# ---------- Flet UI ----------
def main(page: ft.Page):
    page.title = "気象庁 天気予報アプリ（DB版）"
    page.window_width = 900
    page.window_height = 600

    init_db()

    area_data = requests.get(AREA_URL).json()
    offices = area_data["offices"]
    save_areas(offices)

    output = ft.Text("地域を選択してください", size=16)

    def show_weather(area_code, area_name):
        weather_json = requests.get(FORECAST_URL.format(area_code)).json()
        save_forecast(area_code, weather_json)

        rows = load_forecast(area_code)

        text = f"【{area_name}】\n\n"
        for d, w in rows:
            date = datetime.fromisoformat(d).strftime("%Y-%m-%d")
            text += f"{date}：{w}\n"

        output.value = text
        page.update()

    destinations = []
    for code, info in offices.items():
        destinations.append(
            ft.NavigationRailDestination(
                icon=ft.Icons.LOCATION_ON,
                label=info["name"]
            )
        )

    def on_change(e):
        idx = e.control.selected_index
        code = list(offices.keys())[idx]
        name = offices[code]["name"]
        show_weather(code, name)

    rail = ft.NavigationRail(
        destinations=destinations,
        on_change=on_change,
        label_type=ft.NavigationRailLabelType.ALL,
        expand=True,
    )

    page.add(
        ft.Row(
            [
                rail,
                ft.VerticalDivider(width=1),
                ft.Column([output], expand=True)
            ],
            expand=True
        )
    )


# Jupyter notebooks already run an event loop; use the async app runner
await ft.app_async(target=main)
